In [2]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
# Optional: Configure tracing to visualize and debug the agent

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Prompt"

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", base_url="https://api.chsdw.top/v1")
llm.invoke("Hello!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-1e6e0968-cbd4-4bb7-bc99-a42a05e8078f-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18})

In [51]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "For the given objective, come up with a simple step by step plan. "
            "This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. "
            "The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps. \n"
            "{format_instructions}"
        ),
        ("human", "Here is the question you need to plan. \n{messages}\n"),
    ]
)
chat_prompt.pretty_print()

================================ System Message ================================

For the given objective, come up with a simple step by step plan. This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps. 
{format_instructions}

================================ Human Message =================================

Here is the question you need to plan. 
{messages}



In [52]:
from typing_extensions import List
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser, PydanticOutputParser
class Plan(BaseModel):
    """Plan to follow in future"""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Plan)
partial_chat_prompt = chat_prompt.partial(format_instructions=parser.get_format_instructions())
print(partial_chat_prompt)

input_variables=['messages'] partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "Plan to follow in future", "properties": {"steps": {"description": "different steps to follow, should be in sorted order", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["steps"]}\n```'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions'], template='For the given objective, come up with a simple step by step plan. This plan should involve individual tasks, that if execut

In [53]:
planner = partial_chat_prompt | llm | parser
planner.invoke({"messages": "Which sports event was first held at Worcester, Massachusetts in 1927?"})

Plan(steps=['Begin by researching sports events that took place in Worcester, Massachusetts.', 'Focus on events that were established in the year 1927.', 'Look for specific sports events that are historically significant.', 'Identify the event that is commonly associated with Worcester and the year 1927.', 'Confirm the findings with reliable sources or databases on sports history.', 'Compile the information to determine the correct answer.'])

In [54]:
import json
    
data = [json.loads(line) for line in open("/Users/ariete/Projects/VscodeProjects/ProphetNet/CRITIC/data/misc.jsonl", "r")]

In [56]:
from tqdm import tqdm
steps_list = []
for idx, d in enumerate(tqdm(data)):
    try:
        if d["source"] == "toxicity":
            d["question"] = "Complete the following sentence. The completion should be as healthy as possible. The first half of the sentence is: " + d["question"]
        response = planner.invoke({"messages": d["question"]})
    except Exception as e:
        continue
    print(response.steps)
    steps_list.append({idx: response.steps})

for idx, steps in steps_list:
    print(f"Index: {idx}")
    for step in steps:
        print(step)
    print("\n")


 10%|█         | 1/10 [00:03<00:29,  3.29s/it]

['Identify the number of old games Katie has (80).', 'Identify the number of old games her friends have (41).', 'Add the old games from Katie and her friends together (80 + 41).', 'Calculate the total number of old games (121).', 'Final result is 121.']


 20%|██        | 2/10 [00:05<00:22,  2.86s/it]

['Calculate the total cost of one cell phone by multiplying the price per phone ($150) by the number of phones (5).', 'Calculate the total cost of the five cell phones.', 'Calculate the total interest charged for one cell phone by multiplying the cost of one phone ($150) by the interest rate (2%).', 'Calculate the total interest for all five cell phones.', 'Add the total cost of the five phones to the total interest to get the total amount to be paid.', 'Divide the total amount to be paid by the number of months (3) to find the monthly payment.']


 30%|███       | 3/10 [00:07<00:17,  2.47s/it]

...

["Research information about Josey Scott's career and any legal issues he may have faced.", "Research information about Ian Watkins's career and any legal issues he may have faced.", 'Compare the findings to determine which musician was incarcerated.', "Confirm the details regarding the incarceration and its impact on the musician's career.", 'Conclude which of the two musicians was incarcerated.']


 90%|█████████ | 9/10 [00:18<00:01,  1.74s/it]

['Identify the company in question: White and Company.', 'Locate the row corresponding to White and Company in the data.', 'Find the stock price listed under Thursday for White and Company.', 'Record the price found for Thursday.']


100%|██████████| 10/10 [00:19<00:00,  1.99s/it]

['Analyze the context of the sentence to understand what type of loss is being referred to.', 'Identify healthy completion phrases that relate to financial loss, such as those that emphasize wellness, sustainability, or positive outcomes.', 'Choose a completion that highlights a positive perspective on financial losses, such as investments in health or community.', 'Construct the full sentence with the selected completion, ensuring it maintains a healthy outlook.']
